## Парсер сайта dobrotsen.ru

Сделал по быстрому, за 2 часа. Сайт простой, без пагинации. Из требований было только собрать названия, цены и картинки. Поэтому все собрал с грида и запулил в датафрейм.


In [9]:
#начинаем с импорта библиотек
import requests
from requests.auth import HTTPProxyAuth # для проксей
from  bs4 import BeautifulSoup as bs
import csv
from urllib.request import urljoin
from tqdm.notebook import tqdm
from collections import deque
import pandas as pd
from datetime import datetime

In [95]:
# Задаем глобальные параметры
headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'
          }
domen = 'https://dobrotsen.ru'
url='https://dobrotsen.ru/catalog/'
file_product_links = 'dobortsen_product_links.csv'
file_products = 'dobortsen_products.csv'

In [92]:
# Функция сбора ссылок на основные продуктовые категории
def get_category_links(headers, domen, url):
    session = requests.Session()
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'lxml')
    links = []
    cat_links=[]
    urls = set()
    ul = soup.find('div', class_='catalog active sections-in-catalog-main').find_all('a', href=True)
    for u in ul:
        urls.add(u['href'])
    for i in urls:
        link  = urljoin(domen,i)
        links.append(link)
    for i in links:
        ii = i.split('/')
        if len(ii)>=5 & len(ii)<5:
            cat_links.append(i)    
    return cat_links[1:] 

In [98]:
# Фуекция сбора инфы с грида 
def get_all_params(cl, headers, domen):
    params={}
    params['title']=[]
    params['price']=[]
    params['item_img']=[]
    params['item_url']=[]
    for i in tqdm(cl, desc='Getting info from grid', leave=False):
        session = requests.Session()
        r = session.get(i, headers=headers)
        soup = bs(r.text, 'lxml')
        a = soup.find_all('div', class_='products-item')
        for i in a:
            ii = i.find('img', alt=True)['alt']
            ki = i.find('div', class_='products-item-price').text.strip()
            kii =ki[:10].strip()
            f = i.find('img', src=True)['src']
            ff = domen+f
            ji = i.find('a', class_='products-item-image')['href']
            url=domen+ji
            params['title'].append(ii)
            params['price'].append(kii)
            params['item_img'].append(ff)
            params['item_url'].append(url)   
    return params   

In [99]:
# Запуск парсера
cl=get_category_links(headers, domen, url)
pl = get_all_params(cl, headers, domen)

In [100]:
# Загружаем итоги в датафрейм
df = pd.DataFrame(pl)
df.sample(35)

,title,price,item_img,item_url
1486,Хлеб Печеночный Микоян 300г,69.90 ₽,https://dobrotsen.ru/thumb/182x182xffffff/uplo...,https://dobrotsen.ru/catalog/detail/61541_-300...
376,Круассан 7дней Вареная сгущенка/Какао/Ваниль в...,75.90 ₽,https://dobrotsen.ru/thumb/182x182xffffff/uplo...,https://dobrotsen.ru/catalog/detail/165952_-7-...
1317,"Пакеты фасовочные универсальные Тов,Чистов 24*...",24.90 ₽,https://dobrotsen.ru/thumb/182x182xffffff/uplo...,https://dobrotsen.ru/catalog/detail/66017_-24-...
1296,Батарейки солевые Трофи R03-10S ААА 10шт,39 ₽,https://dobrotsen.ru/thumb/182x182xffffff/uplo...,https://dobrotsen.ru/catalog/detail/59163_-r03...
947,"Икра из кабачков ГОСТ 0,5л",28.90 ₽,https://dobrotsen.ru/thumb/182x182xffffff/uplo...,https://dobrotsen.ru/catalog/detail/157864_-0-...
704,Косметический набор 7 days (7+1) Корея 270г,179.90 ₽,https://dobrotsen.ru/thumb/182x182xffffff/uplo...,https://dobrotsen.ru/catalog/detail/169899_-7-...
277,Незамерзающая жидкость зимняя -15С 4л,97.90 ₽,https://dobrotsen.ru/thumb/182x182xffffff/uplo...,https://dobrotsen.ru/catalog/detail/58737_neza...
1618,ЗМЖ Молочносодержащий продукт с наполнителем в...,99.50 ₽,https://dobrotsen.ru/thumb/182x182xffffff/uplo...,https://dobrotsen.ru/catalog/detail/62689_-23-...
672,Гвоздика Челтрейд 10г,14.50 ₽,https://dobrotsen.ru/thumb/182x182xffffff/uplo...,https://dobrotsen.ru/catalog/detail/164163_-10...
515,Крупа в варочных пакетах Пшено шлифованное 5*8...,32.90 ₽,https://dobrotsen.ru/thumb/182x182xffffff/uplo...,https://dobrotsen.ru/catalog/detail/127125_-5-...


In [102]:
# Сколько позиций спарсили
df.shape[0]

1722